In [1]:
import pandas as pd;
import numpy as np;
import nltk;
from nltk import pos_tag, word_tokenize;
from nltk.corpus import stopwords;
from nltk import RegexpParser;

# Load Data

In [2]:
df = pd.read_csv('./winemag-data-130k-v2.csv', index_col=0)
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [3]:
df = df.drop(columns=['designation', 'province', 'region_1', 'region_2', 'taster_twitter_handle'])

In [4]:
df.head()

,country,description,points,price,taster_name,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [ ]:
df.info()

In [5]:
# Drop nulls
df = df.dropna().reset_index(drop=True)
df

,country,description,points,price,taster_name,title,variety,winery
0,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
1,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
2,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
3,US,"Much like the regular bottling from 2012, this...",87,65.0,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
4,Spain,Blackberry and raspberry aromas show a typical...,87,15.0,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
...,...,...,...,...,...,...,...,...
96415,Germany,Notes of honeysuckle and cantaloupe sweeten th...,90,28.0,Anna Lee C. Iijima,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
96416,US,Citation is given as much as a decade of bottl...,90,75.0,Paul Gregutt,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
96417,France,Well-drained gravel soil gives this wine its c...,90,30.0,Roger Voss,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
96418,France,"A dry style of Pinot Gris, this is crisp with ...",90,32.0,Roger Voss,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


## 'points' feature engineering

Every taster may have different preferences and may be more generous with their ratings than others. For instance a rating of 87 points may be average for one taster and superb relative to the average rating of another user. We want to capture the rating of a wine relative to the preferences of a unique taster by baselining each taster's rating relative to their average rating. Additionally, we will rescale the 'points' feature to a 5-star rating system determined each taster's range of ratings given in the data.

In [6]:
tasters = set(df['taster_name'])
tasters

{'Alexander Peartree',
 'Anna Lee C. Iijima',
 'Anne Krebiehl\xa0MW',
 'Carrie Dykes',
 'Christina Pickard',
 'Fiona Adams',
 'Jeff Jenssen',
 'Jim Gordon',
 'Joe Czerwinski',
 'Kerin O’Keefe',
 'Lauren Buzzeo',
 'Matt Kettmann',
 'Michael Schachner',
 'Mike DeSimone',
 'Paul Gregutt',
 'Roger Voss',
 'Sean P. Sullivan',
 'Susan Kostrzewa',
 'Virginie Boone'}

In [7]:
taster_ratings_dict = {'taster': ['mean', '20qt', '40qt', '60qt', '80qt']}
for i in tasters:
    filtered_df = df.copy()[df.copy()['taster_name']==i]
    mean_points = np.round(np.mean(filtered_df['points']),2)
    quantile20 = np.round(np.quantile(filtered_df['points'], 0.20),2)
    quantile40 = np.round(np.quantile(filtered_df['points'], 0.40),2)
    quantile60 = np.round(np.quantile(filtered_df['points'], 0.60),2)
    quantile80 = np.round(np.quantile(filtered_df['points'], 0.80),2)
    taster_ratings_dict.update({i:[mean_points,quantile20,quantile40,quantile60,quantile80]})
taster_ratings_dict
    

{'taster': ['mean', '20qt', '40qt', '60qt', '80qt'],
 'Matt Kettmann': [90.03, 88.0, 89.0, 91.0, 92.0],
 'Paul Gregutt': [89.09, 87.0, 88.0, 90.0, 92.0],
 'Fiona Adams': [86.89, 86.0, 87.0, 87.0, 88.0],
 'Jeff Jenssen': [88.33, 87.0, 88.0, 89.0, 90.0],
 'Mike DeSimone': [89.1, 88.0, 89.0, 90.0, 91.0],
 'Virginie Boone': [89.22, 87.0, 88.0, 90.0, 92.0],
 'Christina Pickard': [87.83, 87.0, 87.0, 89.0, 89.0],
 'Jim Gordon': [88.63, 86.0, 88.0, 89.0, 91.0],
 'Michael Schachner': [86.91, 84.0, 86.0, 88.0, 90.0],
 'Roger Voss': [88.61, 86.0, 88.0, 89.0, 91.0],
 'Anne Krebiehl\xa0MW': [90.61, 89.0, 90.0, 91.0, 93.0],
 'Alexander Peartree': [85.85, 84.0, 86.0, 86.0, 87.0],
 'Joe Czerwinski': [88.54, 86.0, 88.0, 89.0, 91.0],
 'Anna Lee C. Iijima': [88.41, 86.0, 88.0, 89.0, 90.0],
 'Susan Kostrzewa': [86.61, 84.0, 86.0, 87.0, 89.0],
 'Kerin O’Keefe': [88.91, 87.0, 88.0, 89.0, 91.0],
 'Carrie Dykes': [86.38, 85.0, 86.0, 87.0, 88.0],
 'Sean P. Sullivan': [88.76, 87.0, 88.0, 89.0, 91.0],
 'Lauren B

In [8]:
taster_ratings_df = pd.DataFrame.from_dict(taster_ratings_dict)
taster_ratings_df

,taster,Matt Kettmann,Paul Gregutt,Fiona Adams,Jeff Jenssen,Mike DeSimone,Virginie Boone,Christina Pickard,Jim Gordon,Michael Schachner,Roger Voss,Anne Krebiehl MW,Alexander Peartree,Joe Czerwinski,Anna Lee C. Iijima,Susan Kostrzewa,Kerin O’Keefe,Carrie Dykes,Sean P. Sullivan,Lauren Buzzeo
0,mean,90.03,89.09,86.89,88.33,89.1,89.22,87.83,88.63,86.91,88.61,90.61,85.85,88.54,88.41,86.61,88.91,86.38,88.76,87.56
1,20qt,88.00,87.00,86.00,87.00,88.0,87.00,87.00,86.00,84.00,86.00,89.00,84.00,86.00,86.00,84.00,87.00,85.00,87.00,85.00
2,40qt,89.00,88.00,87.00,88.00,89.0,88.00,87.00,88.00,86.00,88.00,90.00,86.00,88.00,88.00,86.00,88.00,86.00,88.00,87.00
3,60qt,91.00,90.00,87.00,89.00,90.0,90.00,89.00,89.00,88.00,89.00,91.00,86.00,89.00,89.00,87.00,89.00,87.00,89.00,88.00
4,80qt,92.00,92.00,88.00,90.00,91.0,92.00,89.00,91.00,90.00,91.00,93.00,87.00,91.00,90.00,89.00,91.00,88.00,91.00,90.00


In [9]:
taster_ratings_df_T = taster_ratings_df.iloc[:,1:].transpose().reset_index().rename(columns=taster_ratings_df.iloc[:,0])
taster_ratings_df_T = taster_ratings_df_T.rename(columns={'index':'taster'})
taster_ratings_df_T.insert(0,'taster_id',[i+1 for i in taster_ratings_df_T.index])
taster_ratings_df_T

,taster_id,taster,mean,20qt,40qt,60qt,80qt
0,1,Matt Kettmann,90.03,88.0,89.0,91.0,92.0
1,2,Paul Gregutt,89.09,87.0,88.0,90.0,92.0
2,3,Fiona Adams,86.89,86.0,87.0,87.0,88.0
3,4,Jeff Jenssen,88.33,87.0,88.0,89.0,90.0
4,5,Mike DeSimone,89.10,88.0,89.0,90.0,91.0
5,6,Virginie Boone,89.22,87.0,88.0,90.0,92.0
6,7,Christina Pickard,87.83,87.0,87.0,89.0,89.0
7,8,Jim Gordon,88.63,86.0,88.0,89.0,91.0
8,9,Michael Schachner,86.91,84.0,86.0,88.0,90.0
9,10,Roger Voss,88.61,86.0,88.0,89.0,91.0


In [10]:
def assign_quantile_rating(row, quantile_table):
    """
    Assign a quantile based on the reviewer's points.

    Parameters:
    - row: Pandas Series representing a row of the wine DataFrame.
    - quantile_table: Pandas DataFrame with quantiles for each reviewer.

    Returns:
    - Quantile value (1 to 5).
    """
    taster = row['taster_name']
    points = row['points']

    if taster in quantile_table['taster'].values:
        quantiles = quantile_table[quantile_table['taster'] == taster].iloc[0, 3:].values
        for i, qt in enumerate(quantiles):
            if points < qt:
                return i + 1  # Return the quantile value (1 to 5)
        return 5  # If points are greater than or equal to the highest quantile, return 5
    else:
        return None  # If taster is not found in the quantile table, return None


In [11]:
# Apply the function to create a new column 'rating' in the 'df' DataFrame
df.insert(3, 'rating',df.apply(assign_quantile_rating, axis=1, quantile_table=taster_ratings_df_T))
df.head()

,country,description,points,rating,price,taster_name,title,variety,winery
0,Portugal,"This is ripe and fruity, a wine that is smooth...",87,2,15.0,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
1,US,"Tart and snappy, the flavors of lime flesh and...",87,2,14.0,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
2,US,"Pineapple rind, lemon pith and orange blossom ...",87,5,13.0,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
3,US,"Much like the regular bottling from 2012, this...",87,2,65.0,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
4,Spain,Blackberry and raspberry aromas show a typical...,87,3,15.0,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem


## Natural Language Processing of 'description' - Creating Taste/Pallette Profile of Each Wine Reviewed

In [12]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/wgemba/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wgemba/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/wgemba/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Using ChatGPT I generated 200 frequently used words that can be used to describe the taste palette of wine.

In [13]:
common_wine_palette_words = [
    'fruity', 'citrus', 'berry', 'cherry', 'plum', 'apple', 'pear', 'peach',
    'apricot', 'tropical', 'blackberry', 'raspberry', 'strawberry', 'currant',
    'black currant', 'blueberry', 'orange', 'lemon', 'lime', 'grapefruit',
    'pineapple', 'melon', 'peel', 'zest', 'jam', 'ripe', 'green', 'underripe',
    'overripe', 'cooked', 'candied', 'dried', 'fresh', 'sweet', 'dry', 'off-dry',
    'crisp', 'tart', 'smooth', 'silky', 'creamy', 'velvety', 'rough', 'astringent',
    'tannic', 'soft', 'structured', 'full-bodied', 'light-bodied', 'medium-bodied',
    'balanced', 'complex', 'simple', 'elegant', 'powerful', 'subtle', 'intense',
    'aromatic', 'fragrant', 'perfumed', 'spicy', 'peppery', 'herbaceous', 'floral',
    'nutty', 'oaky', 'woody', 'smoky', 'earthy', 'minerally', 'salty', 'savory',
    'bright', 'crunchy', 'rich', 'luscious', 'opulent', 'spritzy', 'effervescent',
    'bubbly', 'fresh', 'crisp', 'vibrant', 'juicy', 'zingy', 'lush', 'racy', 'zesty',
    'velvety', 'grippy', 'round', 'sharp', 'spicy', 'tart', 'zippy', 'zesty',
    'structured', 'firm', 'lean', 'creamy', 'lively', 'succulent', 'resinous',
    'silky', 'unctuous', 'robust', 'vigorous', 'full-bodied', 'medium-bodied',
    'light-bodied', 'weighty', 'angular', 'tight', 'broad', 'bold', 'intense',
    'deep', 'lengthy', 'persistent', 'lingering', 'expressive', 'nuanced',
    'layered', 'nuanced', 'focused', 'well-defined', 'balanced', 'harmonious',
    'integrated', 'structured', 'crisp', 'refreshing', 'vivid', 'vibrant',
    'energetic', 'bright', 'uplifting', 'exciting', 'playful', 'youthful',
    'mature', 'aged', 'cellar-worthy', 'drinkable', 'approachable', 'versatile',
    'food-friendly', 'pairable', 'versatile', 'flexible', 'adaptable', 'compatible',
    'cohesive', 'interesting', 'intriguing', 'captivating', 'engaging', 'compelling',
    'memorable', 'remarkable', 'distinctive', 'unique', 'individual', 'characterful',
    'spiced', 'herbal', 'mineral', 'sour', 'bitter', 'sweet', 'umami', 'smoky',
    'toasty', 'buttery', 'creamy', 'vanilla', 'caramel', 'chocolate', 'coffee',
    'spiced', 'nutty', 'floral', 'honey', 'vegetal', 'earthy', 'woody', 'oaky',
    'green', 'fresh', 'savory', 'gamey', 'meaty', 'earthy', 'dusty', 'gritty'
]

In [14]:
'''def extract_qualitative_features(description):
    """
    Extract qualitative features (adjectives and relevant phrases) from a wine description.

    Parameters:
    - description: String, the wine description.

    Returns:
    - List of adjectives and relevant phrases describing qualitative taste features.
    """
    # Tokenize the description
    words = word_tokenize(description)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]

    # Get part-of-speech tags for each word
    pos_tags = pos_tag(filtered_words)

    # Define a grammar for chunking phrases
    grammar = r"""
        NP: {<DT>?<JJ.*>*<NN.*>+}   # Chunk sequences of DT, JJ, NN
        JJ: {<JJ>}
        NNS: {<NNS>}
    """

    # Create a chunk parser with the defined grammar
    chunk_parser = RegexpParser(grammar)

    # Apply chunking to the part-of-speech tagged words
    tree = chunk_parser.parse(pos_tags)

    # Extract adjectives and relevant phrases
    qualitative_features = []
    for subtree in tree.subtrees(filter=lambda x: x.label() in ['JJ', 'NP', 'NNS']):
        if subtree.label() == 'JJ':
            qualitative_features.append(subtree.leaves()[0][0])
        elif subtree.label() == 'NP' or subtree.label() == 'NNS':
            for word, tag in subtree.leaves():
                if tag.startswith('NN') or tag.startswith('JJ'):
                    qualitative_features.append(word)

    # Filter out sentiment-related words
    sentiment_words = set(['good', 'better', 'best', 'excellent', 'great', 'superb', 'delicious', 'fantastic',
                           'bad', 'worse', 'worst', 'poor', 'horrible', 'terrible', 'awful', 'drinkable'])
    
    qualitative_features = [word for word in qualitative_features if word.lower() not in sentiment_words]

    return qualitative_features'''

'def extract_qualitative_features(description):\n    """\n    Extract qualitative features (adjectives and relevant phrases) from a wine description.\n\n    Parameters:\n    - description: String, the wine description.\n\n    Returns:\n    - List of adjectives and relevant phrases describing qualitative taste features.\n    """\n    # Tokenize the description\n    words = word_tokenize(description)\n\n    # Remove stop words\n    stop_words = set(stopwords.words(\'english\'))\n    filtered_words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]\n\n    # Get part-of-speech tags for each word\n    pos_tags = pos_tag(filtered_words)\n\n    # Define a grammar for chunking phrases\n    grammar = r"""\n        NP: {<DT>?<JJ.*>*<NN.*>+}   # Chunk sequences of DT, JJ, NN\n        JJ: {<JJ>}\n        NNS: {<NNS>}\n    """\n\n    # Create a chunk parser with the defined grammar\n    chunk_parser = RegexpParser(grammar)\n\n    # Apply chunking to the part-of-

In [15]:
def extract_qualitative_features(description, common_words):
    """
    Extract qualitative features (words in the common_words list) from a wine description.

    Parameters:
    - description: String, the wine description.
    - common_words: List, a list of common words associated with the taste palette of wine.

    Returns:
    - List of words describing qualitative features found in common_words.
    """
    # Tokenize the description
    words = word_tokenize(description)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]

    # Get part-of-speech tags for each word
    pos_tags = pos_tag(filtered_words)

    # Define a grammar for chunking phrases
    grammar = r"""
        NP: {<DT>?<JJ.*>*<NN.*>+}   # Chunk sequences of DT, JJ, NN
        JJ: {<JJ>}
        NNS: {<NNS>}
    """

    # Create a chunk parser with the defined grammar
    chunk_parser = RegexpParser(grammar)

    # Apply chunking to the part-of-speech tagged words
    tree = chunk_parser.parse(pos_tags)

    # Extract words in the common_words list
    qualitative_features = []
    for subtree in tree.subtrees(filter=lambda x: x.label() in ['JJ', 'NP', 'NNS']):
        for word, tag in subtree.leaves():
            if tag.startswith('NN') or tag.startswith('JJ'):
                if word.lower() in common_words:
                    qualitative_features.append(word.lower())

    return qualitative_features

In [16]:
extract_qualitative_features(df['description'][0], common_wine_palette_words)

['ripe', 'fruity', 'smooth', 'firm', 'juicy', 'berry', 'drinkable']

In [18]:
taste_profiles = []
for desc in df['description']:
    profile = extract_qualitative_features(desc, common_wine_palette_words)
    taste_profiles.append(profile)

taste_profiles[:10]

[['ripe', 'fruity', 'smooth', 'firm', 'juicy', 'berry', 'drinkable'],
 ['tart', 'green', 'pineapple'],
 ['pineapple', 'lemon', 'orange', 'opulent', 'astringent'],
 ['rough', 'tannic', 'earthy', 'herbal'],
 ['blackberry', 'green', 'spicy', 'herbal', 'plum', 'fresh'],
 ['bright', 'berry', 'savory', 'fresh', 'soft'],
 ['dry', 'firm'],
 ['savory', 'peach', 'fruity', 'fresh', 'elegant'],
 ['fresh', 'apple', 'dry', 'crisp'],
 ['soft', 'plum', 'oaky', 'coffee', 'chocolate']]

In [19]:
#wines_df = df.copy()
df.insert(2, 'taste_profile', taste_profiles)
df.head()

,country,description,taste_profile,points,rating,price,taster_name,title,variety,winery
0,Portugal,"This is ripe and fruity, a wine that is smooth...","[ripe, fruity, smooth, firm, juicy, berry, dri...",87,2,15.0,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
1,US,"Tart and snappy, the flavors of lime flesh and...","[tart, green, pineapple]",87,2,14.0,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
2,US,"Pineapple rind, lemon pith and orange blossom ...","[pineapple, lemon, orange, opulent, astringent]",87,5,13.0,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
3,US,"Much like the regular bottling from 2012, this...","[rough, tannic, earthy, herbal]",87,2,65.0,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
4,Spain,Blackberry and raspberry aromas show a typical...,"[blackberry, green, spicy, herbal, plum, fresh]",87,3,15.0,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem


In [20]:
wine_id_dict = {}
wines = list(set(df.title))

for i in range(len(wines)):
    wine_id_dict.update({wines[i]:i+1})

In [21]:
df.insert(0,'wine_id',df['title'].map(wine_id_dict))
df.head()

,wine_id,country,description,taste_profile,points,rating,price,taster_name,title,variety,winery
0,19387,Portugal,"This is ripe and fruity, a wine that is smooth...","[ripe, fruity, smooth, firm, juicy, berry, dri...",87,2,15.0,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
1,29816,US,"Tart and snappy, the flavors of lime flesh and...","[tart, green, pineapple]",87,2,14.0,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
2,34836,US,"Pineapple rind, lemon pith and orange blossom ...","[pineapple, lemon, orange, opulent, astringent]",87,5,13.0,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
3,72740,US,"Much like the regular bottling from 2012, this...","[rough, tannic, earthy, herbal]",87,2,65.0,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
4,28842,Spain,Blackberry and raspberry aromas show a typical...,"[blackberry, green, spicy, herbal, plum, fresh]",87,3,15.0,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem


In [134]:
# Add taster_id
taster_id_dict = {}
for i,j in zip(taster_ratings_df_T['taster_id'], taster_ratings_df_T['taster']):
    taster_id_dict.update({j:i})

taster_id_dict

{'Matt Kettmann': 1,
 'Paul Gregutt': 2,
 'Fiona Adams': 3,
 'Jeff Jenssen': 4,
 'Mike DeSimone': 5,
 'Virginie Boone': 6,
 'Christina Pickard': 7,
 'Jim Gordon': 8,
 'Michael Schachner': 9,
 'Roger Voss': 10,
 'Anne Krebiehl\xa0MW': 11,
 'Alexander Peartree': 12,
 'Joe Czerwinski': 13,
 'Anna Lee C. Iijima': 14,
 'Susan Kostrzewa': 15,
 'Kerin O’Keefe': 16,
 'Carrie Dykes': 17,
 'Sean P. Sullivan': 18,
 'Lauren Buzzeo': 19}

In [108]:
df.insert(8,'taster_id',df['taster_name'].map(taster_id_dict))
df.head()

,wine_id,country,description,taste_profile,points,rating,price,taster_name,taster_id,title,variety,winery
0,19387,Portugal,"This is ripe and fruity, a wine that is smooth...","[ripe, fruity, smooth, firm, juicy, berry, dri...",87,2,15.0,Roger Voss,10,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
1,29816,US,"Tart and snappy, the flavors of lime flesh and...","[tart, green, pineapple]",87,2,14.0,Paul Gregutt,2,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
2,34836,US,"Pineapple rind, lemon pith and orange blossom ...","[pineapple, lemon, orange, opulent, astringent]",87,5,13.0,Alexander Peartree,12,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
3,72740,US,"Much like the regular bottling from 2012, this...","[rough, tannic, earthy, herbal]",87,2,65.0,Paul Gregutt,2,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
4,28842,Spain,Blackberry and raspberry aromas show a typical...,"[blackberry, green, spicy, herbal, plum, fresh]",87,3,15.0,Michael Schachner,9,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem


# Create taster rating dataset

In [142]:
taster_ratings_y = df.copy()[['taster_id','wine_id','rating']]
taster_ratings_y = taster_ratings_y.sort_values(by='taster_id').reset_index(drop=True)
taster_ratings_y

,taster_id,wine_id,rating
0,1,611,3
1,1,10892,1
2,1,16715,3
3,1,85145,1
4,1,41043,5
...,...,...,...
96415,19,8604,1
96416,19,35194,1
96417,19,39165,1
96418,19,56834,4


# Create Wines Ratings Dataset

## Aggregate by wine_id

In [65]:
wines_df = df.copy()
wines_df = wines_df[['wine_id','country','taste_profile','rating','price','title','variety','winery']]
wines_df

,wine_id,country,taste_profile,rating,price,title,variety,winery
0,19387,Portugal,"[ripe, fruity, smooth, firm, juicy, berry, dri...",2,15.0,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
1,29816,US,"[tart, green, pineapple]",2,14.0,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
2,34836,US,"[pineapple, lemon, orange, opulent, astringent]",5,13.0,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
3,72740,US,"[rough, tannic, earthy, herbal]",2,65.0,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
4,28842,Spain,"[blackberry, green, spicy, herbal, plum, fresh]",3,15.0,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
...,...,...,...,...,...,...,...,...
96415,10944,Germany,"[juicy, grapefruit, honey, peach]",5,28.0,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
96416,56223,US,"[cherry, soft]",4,75.0,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
96417,67138,France,"[crisp, dry, ripe, fruity]",4,30.0,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
96418,43001,France,"[dry, crisp, powerful, apple]",4,32.0,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


In [66]:
# Ensure 'taste_profile' column is a list for each row
#wines_df['taste_profile'] = wines_df['taste_profile'].apply(lambda x: [] if pd.isna(x) else x)

# Concatenate the 'taste_profile' lists into a union set
wines_df= wines_df.groupby('wine_id').agg({
    'country': 'first',
    'taste_profile': lambda x: list(set().union(*(set(profile) for profile in x))) if any(x) else [],
    'rating': 'mean',
    'price': 'mean',
    'title': 'first',
    'variety': 'first',
    'winery': 'first',
}).reset_index()

# Display the resulting DataFrame
wines_df

,wine_id,country,taste_profile,rating,price,title,variety,winery
0,1,Germany,"[sweet, apple, spicy, ripe, citrus]",5.0,31.0,S.A. Prüm 2009 Graacher Himmelreich Spätlese R...,Riesling,S.A. Prüm
1,2,France,"[currant, soft]",2.0,24.0,Château Haut Peyrefaure 2015 Blaye Côtes de B...,Bordeaux-style Red Blend,Château Haut Peyrefaure
2,3,Austria,"[luscious, zest, juicy, tart, citrus, apricot]",5.0,45.0,Kracher 2012 Nouvelle Vague Nr. 6 Grande Cuvée...,White Blend,Kracher
3,4,US,"[cherry, coffee, green, vanilla]",5.0,35.0,Bridge Press 2012 Cabernet Sauvignon (Walla Wa...,Cabernet Sauvignon,Bridge Press
4,5,Italy,"[coffee, astringent]",1.0,65.0,Cennatoio 2011 Etrusco Sangiovese (Toscana),Sangiovese,Cennatoio
...,...,...,...,...,...,...,...,...
87499,87500,US,"[sweet, apple, ripe, racy]",1.0,19.0,Sofia 2014 Blanc de Blancs Sparkling (Monterey...,Sparkling Blend,Sofia
87500,87501,France,"[firm, orange, citrus]",5.0,27.0,Cave de Kientzheim-Kaysersberg 2012 Anne de K ...,Riesling,Cave de Kientzheim-Kaysersberg
87501,87502,Argentina,"[cherry, deep, bold, dusty, bright, juicy, berry]",5.0,140.0,Achaval-Ferrer 2012 Finca Bella Vista Malbec (...,Malbec,Achaval-Ferrer
87502,87503,US,"[spritzy, fresh, aromatic]",3.0,15.0,Siltstone 2014 Blacktail Vineyard Pinot Gris (...,Pinot Gris,Siltstone


In [68]:
wines_df = wines_df.rename(columns={'rating':'avg_rating', 'price':'avg_price'})
wines_df.head(3)

,wine_id,country,taste_profile,avg_rating,avg_price,title,variety,winery
0,1,Germany,"[sweet, apple, spicy, ripe, citrus]",5.0,31.0,S.A. Prüm 2009 Graacher Himmelreich Spätlese R...,Riesling,S.A. Prüm
1,2,France,"[currant, soft]",2.0,24.0,Château Haut Peyrefaure 2015 Blaye Côtes de B...,Bordeaux-style Red Blend,Château Haut Peyrefaure
2,3,Austria,"[luscious, zest, juicy, tart, citrus, apricot]",5.0,45.0,Kracher 2012 Nouvelle Vague Nr. 6 Grande Cuvée...,White Blend,Kracher


## One Hot Encoding of 'taste_profile' 

In [69]:
# Extract unique taste profile items
unique_taste_items = set(item for sublist in wines_df['taste_profile'] for item in sublist)
unique_taste_items

{'aged',
 'angular',
 'apple',
 'approachable',
 'apricot',
 'aromatic',
 'astringent',
 'balanced',
 'berry',
 'bitter',
 'blackberry',
 'blueberry',
 'bold',
 'bright',
 'broad',
 'bubbly',
 'buttery',
 'candied',
 'captivating',
 'caramel',
 'characterful',
 'cherry',
 'chocolate',
 'citrus',
 'coffee',
 'cohesive',
 'compatible',
 'compelling',
 'complex',
 'cooked',
 'creamy',
 'crisp',
 'crunchy',
 'currant',
 'deep',
 'distinctive',
 'dried',
 'drinkable',
 'dry',
 'dusty',
 'earthy',
 'effervescent',
 'elegant',
 'energetic',
 'engaging',
 'exciting',
 'expressive',
 'firm',
 'flexible',
 'floral',
 'focused',
 'fragrant',
 'fresh',
 'fruity',
 'gamey',
 'grapefruit',
 'green',
 'grippy',
 'gritty',
 'harmonious',
 'herbaceous',
 'herbal',
 'honey',
 'individual',
 'integrated',
 'intense',
 'interesting',
 'intriguing',
 'jam',
 'juicy',
 'layered',
 'lean',
 'lemon',
 'lengthy',
 'lime',
 'lingering',
 'lively',
 'luscious',
 'lush',
 'mature',
 'meaty',
 'melon',
 'memorable

In [70]:
# Create one-hot-encoded columns for each taste profile item
for item in unique_taste_items:
    wines_df[item] = wines_df['taste_profile'].apply(lambda x: 1 if item in x else 0)

# Drop the original 'taste_profile' column as it's no longer needed
wines_df = wines_df.drop('taste_profile', axis=1)

wines_df

/var/folders/20/g71q1f5n1ml1vv9pkb85ps000000gn/T/ipykernel_13198/3852433838.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  wines_df[item] = wines_df['taste_profile'].apply(lambda x: 1 if item in x else 0)
/var/folders/20/g71q1f5n1ml1vv9pkb85ps000000gn/T/ipykernel_13198/3852433838.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  wines_df[item] = wines_df['taste_profile'].apply(lambda x: 1 if item in x else 0)
/var/folders/20/g71q1f5n1ml1vv9pkb85ps000000gn/T/ipykernel_13198/3852433838.py:3: PerformanceWarning: DataFrame is

,wine_id,country,avg_rating,avg_price,title,variety,winery,astringent,bubbly,fragrant,...,aromatic,characterful,approachable,sharp,gamey,lingering,round,remarkable,fresh,dry
0,1,Germany,5.0,31.0,S.A. Prüm 2009 Graacher Himmelreich Spätlese R...,Riesling,S.A. Prüm,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,France,2.0,24.0,Château Haut Peyrefaure 2015 Blaye Côtes de B...,Bordeaux-style Red Blend,Château Haut Peyrefaure,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Austria,5.0,45.0,Kracher 2012 Nouvelle Vague Nr. 6 Grande Cuvée...,White Blend,Kracher,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,US,5.0,35.0,Bridge Press 2012 Cabernet Sauvignon (Walla Wa...,Cabernet Sauvignon,Bridge Press,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Italy,1.0,65.0,Cennatoio 2011 Etrusco Sangiovese (Toscana),Sangiovese,Cennatoio,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87499,87500,US,1.0,19.0,Sofia 2014 Blanc de Blancs Sparkling (Monterey...,Sparkling Blend,Sofia,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87500,87501,France,5.0,27.0,Cave de Kientzheim-Kaysersberg 2012 Anne de K ...,Riesling,Cave de Kientzheim-Kaysersberg,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87501,87502,Argentina,5.0,140.0,Achaval-Ferrer 2012 Finca Bella Vista Malbec (...,Malbec,Achaval-Ferrer,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87502,87503,US,3.0,15.0,Siltstone 2014 Blacktail Vineyard Pinot Gris (...,Pinot Gris,Siltstone,0,0,0,...,1,0,0,0,0,0,0,0,1,0


## One Hot Encoding of 'country' and 'variety'

In [30]:
from sklearn.preprocessing import OneHotEncoder;

In [31]:
one_hot = OneHotEncoder(sparse=False, drop='first')

In [71]:
# Reshape the 'country' column to a 2D array as OneHotEncoder expects
countries_reshaped = wines_df['country'].values.reshape(-1, 1)

# Fit and transform the OneHotEncoder on the 'country' column
country_encoded = one_hot.fit_transform(countries_reshaped)

# Create a DataFrame with the encoded values
encoded_df = pd.DataFrame(country_encoded, columns=one_hot.get_feature_names_out(['country']))

# Concatenate the original DataFrame with the encoded DataFrame
wines_df = pd.concat([wines_df, encoded_df], axis=1)

# Drop the original 'country' column as it's no longer needed
wines_df = wines_df.drop('country', axis=1)

wines_df.head()


/Users/wgemba/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,wine_id,avg_rating,avg_price,title,variety,winery,astringent,bubbly,fragrant,zippy,...,country_Serbia,country_Slovakia,country_Slovenia,country_South Africa,country_Spain,country_Switzerland,country_Turkey,country_US,country_Ukraine,country_Uruguay
0,1,5.0,31.0,S.A. Prüm 2009 Graacher Himmelreich Spätlese R...,Riesling,S.A. Prüm,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,2.0,24.0,Château Haut Peyrefaure 2015 Blaye Côtes de B...,Bordeaux-style Red Blend,Château Haut Peyrefaure,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,5.0,45.0,Kracher 2012 Nouvelle Vague Nr. 6 Grande Cuvée...,White Blend,Kracher,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,5.0,35.0,Bridge Press 2012 Cabernet Sauvignon (Walla Wa...,Cabernet Sauvignon,Bridge Press,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5,1.0,65.0,Cennatoio 2011 Etrusco Sangiovese (Toscana),Sangiovese,Cennatoio,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
# Reshape the 'variety' column to a 2D array as OneHotEncoder expects
variety_reshaped = wines_df['variety'].values.reshape(-1, 1)

# Fit and transform the OneHotEncoder on the 'country' column
variety_encoded = one_hot.fit_transform(variety_reshaped)

# Create a DataFrame with the encoded values
encoded_df = pd.DataFrame(variety_encoded, columns=one_hot.get_feature_names_out(['variety']))

# Concatenate the original DataFrame with the encoded DataFrame
wines_df = pd.concat([wines_df, encoded_df], axis=1)

# Drop the original 'variety' column as it's no longer needed
wines_df = wines_df.drop('variety', axis=1)

wines_df.head()

/Users/wgemba/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,wine_id,avg_rating,avg_price,title,winery,astringent,bubbly,fragrant,zippy,pineapple,...,variety_Xynisteri,variety_Yapincak,variety_Zibibbo,variety_Zierfandler,variety_Zierfandler-Rotgipfler,variety_Zinfandel,variety_Zlahtina,variety_Zweigelt,variety_Çalkarası,variety_Žilavka
0,1,5.0,31.0,S.A. Prüm 2009 Graacher Himmelreich Spätlese R...,S.A. Prüm,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,2.0,24.0,Château Haut Peyrefaure 2015 Blaye Côtes de B...,Château Haut Peyrefaure,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,5.0,45.0,Kracher 2012 Nouvelle Vague Nr. 6 Grande Cuvée...,Kracher,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,5.0,35.0,Bridge Press 2012 Cabernet Sauvignon (Walla Wa...,Bridge Press,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1.0,65.0,Cennatoio 2011 Etrusco Sangiovese (Toscana),Cennatoio,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
# Drop 'title' and 'winery' as they are too fragmented
wines_df = wines_df.drop(columns=['title','winery'])

,wine_id,avg_rating,avg_price,astringent,bubbly,fragrant,zippy,pineapple,strawberry,crisp,...,variety_Xynisteri,variety_Yapincak,variety_Zibibbo,variety_Zierfandler,variety_Zierfandler-Rotgipfler,variety_Zinfandel,variety_Zlahtina,variety_Zweigelt,variety_Çalkarası,variety_Žilavka
0,1,5.0,31.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,2.0,24.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,5.0,45.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,5.0,35.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1.0,65.0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
# Select all columns starting after third
columns_to_convert_type = list(wines_df.iloc[:,3:].columns)
columns_to_convert_type[:5]

['astringent', 'bubbly', 'fragrant', 'zippy', 'pineapple']

In [100]:
# Convert columns to int64 type
wines_df[columns_to_convert_type] = wines_df[columns_to_convert_type].astype('int64')

In [103]:
wines_df

,wine_id,avg_rating,avg_price,astringent,bubbly,fragrant,zippy,pineapple,strawberry,crisp,...,variety_Xynisteri,variety_Yapincak,variety_Zibibbo,variety_Zierfandler,variety_Zierfandler-Rotgipfler,variety_Zinfandel,variety_Zlahtina,variety_Zweigelt,variety_Çalkarası,variety_Žilavka
0,1,5.0,31.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,2.0,24.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,5.0,45.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,5.0,35.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1.0,65.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87499,87500,1.0,19.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87500,87501,5.0,27.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87501,87502,5.0,140.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87502,87503,3.0,15.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
wines_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87504 entries, 0 to 87503
Columns: 848 entries, wine_id to variety_Žilavka
dtypes: float64(2), int64(846)
memory usage: 566.8 MB


# Create Taster Dataset

In [109]:
taster_df = df.copy()
taster_df = taster_df[['taster_id','taste_profile','country','rating','price','variety']]
taster_df

,taster_id,taste_profile,country,rating,price,variety
0,10,"[ripe, fruity, smooth, firm, juicy, berry, dri...",Portugal,2,15.0,Portuguese Red
1,2,"[tart, green, pineapple]",US,2,14.0,Pinot Gris
2,12,"[pineapple, lemon, orange, opulent, astringent]",US,5,13.0,Riesling
3,2,"[rough, tannic, earthy, herbal]",US,2,65.0,Pinot Noir
4,9,"[blackberry, green, spicy, herbal, plum, fresh]",Spain,3,15.0,Tempranillo-Merlot
...,...,...,...,...,...,...
96415,14,"[juicy, grapefruit, honey, peach]",Germany,5,28.0,Riesling
96416,2,"[cherry, soft]",US,4,75.0,Pinot Noir
96417,10,"[crisp, dry, ripe, fruity]",France,4,30.0,Gewürztraminer
96418,10,"[dry, crisp, powerful, apple]",France,4,32.0,Pinot Gris


In [131]:
# Calculate number of reviews per taster
tasters_count = taster_df.copy().groupby(by = 'taster_id').size().sort_values(ascending=False)
tasters_count

taster_id
10    20172
9     14944
16     9874
6      9507
2      9497
1      6237
13     5010
18     4925
14     4369
8      4171
11     3389
19     1712
15     1068
5       492
4       469
12      413
17      138
3        27
7         6
dtype: int64

In [136]:
# Create taster_id_count_dict
taster_id_count_dict = {}
for i,j in zip(tasters_count.index, tasters_count.values):
    taster_id_count_dict.update({i:j})

taster_id_count_dict

{10: 20172,
 9: 14944,
 16: 9874,
 6: 9507,
 2: 9497,
 1: 6237,
 13: 5010,
 18: 4925,
 14: 4369,
 8: 4171,
 11: 3389,
 19: 1712,
 15: 1068,
 5: 492,
 4: 469,
 12: 413,
 17: 138,
 3: 27,
 7: 6}

In [110]:
# Step 1: Explode the 'taste_profile' lists to separate rows for each taste word
taster_df_exploded = taster_df.explode('taste_profile')
taster_df_exploded

,taster_id,taste_profile,country,rating,price,variety
0,10,ripe,Portugal,2,15.0,Portuguese Red
0,10,fruity,Portugal,2,15.0,Portuguese Red
0,10,smooth,Portugal,2,15.0,Portuguese Red
0,10,firm,Portugal,2,15.0,Portuguese Red
0,10,juicy,Portugal,2,15.0,Portuguese Red
...,...,...,...,...,...,...
96418,10,powerful,France,4,32.0,Pinot Gris
96418,10,apple,France,4,32.0,Pinot Gris
96419,10,rich,France,4,21.0,Gewürztraminer
96419,10,intense,France,4,21.0,Gewürztraminer


In [111]:
# Step 2: Group by 'taster_id' and 'taste_profile' to calculate the average rating per taste word
average_rating_per_taste_word = taster_df_exploded.groupby(['taster_id', 'taste_profile'])['rating'].mean().reset_index()
average_rating_per_taste_word

,taster_id,taste_profile,rating
0,1,aged,3.500000
1,1,angular,3.666667
2,1,apple,2.758865
3,1,approachable,3.464646
4,1,apricot,2.975610
...,...,...,...
2208,19,youthful,4.500000
2209,19,zest,3.333333
2210,19,zesty,3.333333
2211,19,zingy,3.000000


In [114]:
# Step 3: Group by 'taster_id' and 'country' to calculate the average rating per country
average_rating_per_country = taster_df.groupby(['taster_id', 'country'])['rating'].mean().reset_index()
average_rating_per_country

,taster_id,country,rating
0,1,Chile,3.000000
1,1,US,3.321039
2,2,Canada,3.521739
3,2,France,3.352941
4,2,Italy,2.500000
...,...,...,...
104,19,Israel,3.015152
105,19,Portugal,1.000000
106,19,South Africa,3.519337
107,19,Spain,1.000000


In [115]:
# Step 4: Group by 'taster_id' and 'variety' to calculate the average rating per variety
average_rating_per_variety = df.groupby(['taster_id', 'variety'])['rating'].mean().reset_index()
average_rating_per_variety

,taster_id,variety,rating
0,1,Aglianico,4.666667
1,1,Albariño,2.641026
2,1,Alicante Bouschet,2.500000
3,1,Aligoté,4.000000
4,1,Alvarinho,1.000000
...,...,...,...
1787,19,Tinta Barroca,5.000000
1788,19,Viognier,2.730769
1789,19,Viognier-Grenache Blanc,3.000000
1790,19,White Blend,3.527273


In [123]:
# Step 5: Group by 'taster_id' to calculate the average rating, average price per taster
average_taster_attributes = taster_df.groupby('taster_id').agg({
    'rating': 'mean',
    'price': 'mean',
}).reset_index()

average_taster_attributes = average_taster_attributes.rename(columns={'rating':'avg_rating', 'price':'avg_price'})
average_taster_attributes

,taster_id,avg_rating,avg_price
0,1,3.320988,38.642136
1,2,3.189218,33.644835
2,3,3.407407,31.148148
3,4,3.298507,22.313433
4,5,3.150407,28.132114
5,6,3.288419,46.621963
6,7,3.666667,29.333333
7,8,3.310717,26.935507
8,9,3.150027,25.230929
9,10,3.189570,38.649960


In [124]:
# Step 6: Pivot the DataFrames
pivot_taste_word = average_rating_per_taste_word.pivot(index='taster_id', columns='taste_profile', values='rating').reset_index()
pivot_country = average_rating_per_country.pivot(index='taster_id', columns='country', values='rating').reset_index()
pivot_variety = average_rating_per_variety.pivot(index='taster_id', columns='variety', values='rating').reset_index()

In [128]:
# Step 7: Merge the pivoted DataFrames with the 'average_taster_attributes'
result_taster_df = pd.merge(average_taster_attributes, pivot_taste_word, on='taster_id', how='left')
result_taster_df = pd.merge(result_taster_df, pivot_country, on='taster_id', how='left')
result_taster_df = pd.merge(result_taster_df, pivot_variety, on='taster_id', how='left')

result_taster_df

,taster_id,avg_rating,avg_price,aged,angular,apple,approachable,apricot,aromatic,astringent,...,Xynisteri,Yapincak,Zibibbo,Zierfandler,Zierfandler-Rotgipfler,Zinfandel,Zlahtina,Zweigelt,Çalkarası,Žilavka
0,1,3.320988,38.642136,3.500000,3.666667,2.758865,3.464646,2.975610,3.778947,1.000000,...,NaN,NaN,NaN,NaN,NaN,2.802326,NaN,NaN,NaN,NaN
1,2,3.189218,33.644835,3.500000,2.250000,3.126394,2.980769,3.787879,3.906883,2.507042,...,NaN,NaN,NaN,NaN,NaN,2.646667,NaN,1.500000,NaN,NaN
2,3,3.407407,31.148148,NaN,NaN,3.222222,NaN,4.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,3.298507,22.313433,NaN,NaN,3.327731,NaN,4.068966,5.000000,2.750000,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,5.000000,NaN,3.0
4,5,3.150407,28.132114,NaN,NaN,2.446429,5.000000,1.789474,1.000000,3.000000,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
5,6,3.288419,46.621963,3.187500,3.666667,3.525316,3.104418,3.173077,3.354545,1.142857,...,NaN,NaN,NaN,NaN,NaN,2.914894,NaN,NaN,NaN,NaN
6,7,3.666667,29.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,3.310717,26.935507,4.500000,5.000000,2.913649,3.125000,3.241379,3.846154,3.218750,...,NaN,NaN,NaN,NaN,NaN,3.569124,NaN,NaN,NaN,NaN
8,9,3.150027,25.230929,3.250000,2.648649,2.994985,3.885714,3.306748,3.114583,2.449612,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,3.189570,38.649960,4.125000,2.250000,3.090179,2.845070,3.348754,3.017341,3.000000,...,NaN,NaN,NaN,3.5,NaN,NaN,NaN,2.721311,NaN,NaN


In [129]:
result_taster_df = result_taster_df.fillna(0)
result_taster_df

,taster_id,avg_rating,avg_price,aged,angular,apple,approachable,apricot,aromatic,astringent,...,Xynisteri,Yapincak,Zibibbo,Zierfandler,Zierfandler-Rotgipfler,Zinfandel,Zlahtina,Zweigelt,Çalkarası,Žilavka
0,1,3.320988,38.642136,3.500000,3.666667,2.758865,3.464646,2.975610,3.778947,1.000000,...,0.0,0.0,0.0,0.0,0.0,2.802326,0.0,0.000000,0.0,0.0
1,2,3.189218,33.644835,3.500000,2.250000,3.126394,2.980769,3.787879,3.906883,2.507042,...,0.0,0.0,0.0,0.0,0.0,2.646667,0.0,1.500000,0.0,0.0
2,3,3.407407,31.148148,0.000000,0.000000,3.222222,0.000000,4.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
3,4,3.298507,22.313433,0.000000,0.000000,3.327731,0.000000,4.068966,5.000000,2.750000,...,0.0,0.0,0.0,0.0,0.0,0.000000,3.0,5.000000,0.0,3.0
4,5,3.150407,28.132114,0.000000,0.000000,2.446429,5.000000,1.789474,1.000000,3.000000,...,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.0,0.0
5,6,3.288419,46.621963,3.187500,3.666667,3.525316,3.104418,3.173077,3.354545,1.142857,...,0.0,0.0,0.0,0.0,0.0,2.914894,0.0,0.000000,0.0,0.0
6,7,3.666667,29.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
7,8,3.310717,26.935507,4.500000,5.000000,2.913649,3.125000,3.241379,3.846154,3.218750,...,0.0,0.0,0.0,0.0,0.0,3.569124,0.0,0.000000,0.0,0.0
8,9,3.150027,25.230929,3.250000,2.648649,2.994985,3.885714,3.306748,3.114583,2.449612,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
9,10,3.189570,38.649960,4.125000,2.250000,3.090179,2.845070,3.348754,3.017341,3.000000,...,0.0,0.0,0.0,3.5,0.0,0.000000,0.0,2.721311,0.0,0.0


In [137]:
result_taster_df.insert(1,'num_reviews',result_taster_df['taster_id'].map(taster_id_count_dict))
result_taster_df

,taster_id,num_reviews,avg_rating,avg_price,aged,angular,apple,approachable,apricot,aromatic,...,Xynisteri,Yapincak,Zibibbo,Zierfandler,Zierfandler-Rotgipfler,Zinfandel,Zlahtina,Zweigelt,Çalkarası,Žilavka
0,1,6237,3.320988,38.642136,3.500000,3.666667,2.758865,3.464646,2.975610,3.778947,...,0.0,0.0,0.0,0.0,0.0,2.802326,0.0,0.000000,0.0,0.0
1,2,9497,3.189218,33.644835,3.500000,2.250000,3.126394,2.980769,3.787879,3.906883,...,0.0,0.0,0.0,0.0,0.0,2.646667,0.0,1.500000,0.0,0.0
2,3,27,3.407407,31.148148,0.000000,0.000000,3.222222,0.000000,4.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
3,4,469,3.298507,22.313433,0.000000,0.000000,3.327731,0.000000,4.068966,5.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,3.0,5.000000,0.0,3.0
4,5,492,3.150407,28.132114,0.000000,0.000000,2.446429,5.000000,1.789474,1.000000,...,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.0,0.0
5,6,9507,3.288419,46.621963,3.187500,3.666667,3.525316,3.104418,3.173077,3.354545,...,0.0,0.0,0.0,0.0,0.0,2.914894,0.0,0.000000,0.0,0.0
6,7,6,3.666667,29.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
7,8,4171,3.310717,26.935507,4.500000,5.000000,2.913649,3.125000,3.241379,3.846154,...,0.0,0.0,0.0,0.0,0.0,3.569124,0.0,0.000000,0.0,0.0
8,9,14944,3.150027,25.230929,3.250000,2.648649,2.994985,3.885714,3.306748,3.114583,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
9,10,20172,3.189570,38.649960,4.125000,2.250000,3.090179,2.845070,3.348754,3.017341,...,0.0,0.0,0.0,3.5,0.0,0.000000,0.0,2.721311,0.0,0.0


In [130]:
result_taster_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Columns: 850 entries, taster_id to Žilavka
dtypes: float64(849), int64(1)
memory usage: 126.3 KB


# Export dataframes

In [143]:
# Export to CSV
#taster_ratings_y.to_csv('./taster_ratings_y.csv', index=False)
#wines_df.to_csv('./wines_train.csv',index=False)
#result_taster_df.to_csv('./taster_train.csv', index=False)